# About the notebook
- segmentation_models_pytorch starter code for the competition
- GroupKFold 5 folds
- EfficientnetB3 as encoder Feature Pyramid Network (FPN) as decoder
- Wandb.ai
    - Pytorch W&B Usage Examples from https://docs.wandb.ai/guides/integrations/pytorch
- Inference notebook is coming soon


If this notebook is helpful, feel free to upvote!

In [ ]:
# Install SMP
!pip install segmentation_models_pytorch -q
!pip install timm -q

## Import necessary libraries

In [ ]:
# ====================================================
# Libraries
# ====================================================
import os
import sys
import cv2
import pdb
import time
import warnings
warnings.filterwarnings("ignore")

import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import GroupKFold

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, sampler
from torch.utils.data.sampler import SequentialSampler, RandomSampler

import timm
import segmentation_models_pytorch as smp
from matplotlib import pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2

def seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    print('Done seeding.')
    
seed(42)

# Config

In [ ]:
# ====================================================
# CFG
# ====================================================
class CFG:
    apex=True
    debug=False
    num_workers=4
    model_name='effb3+FPN'
    encoder_name='timm-efficientnet-b3'
    decoder_name='FPN'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=5
    T_max=3 
    lr=1e-4
    min_lr=1e-6
    batch_size=32
    image_size = [384, 384]
    mean=timm.data.IMAGENET_DEFAULT_MEAN
    std=timm.data.IMAGENET_DEFAULT_STD
    seed=42
    n_fold=5
    trn_fold=[0] # You go with [0, 1, 2, 3, 4] if you want to train for 5 folds
    train=True,
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    sample_submission='../input/sartorius-cell-instance-segmentation/sample_submission.csv'
    train_df_path="../input/sartorius-cell-instance-segmentation/train.csv"
    train_base="../input/sartorius-cell-instance-segmentation/train"
    test_base="../input/sartorius-cell-instance-segmentation/test"
    

### Logging to Wandb.ai

In [ ]:
# 1-) Go to wandb.ai/authorize copy the api key
# 2-) From the top menu click Add-ons > secrets > add a new secret > name 'Label' as wandb and paste your api key to value
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()
#wandb_api = user_secrets.get_secret("wandb")

In [ ]:
#import wandb
#wandb.login(key=wandb_api)

#def class2dict(f):
#    return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

#run = wandb.init(project="sartarious-segmentation-exps", 
#                 name="exp1",
#                 config=class2dict(CFG),
#                 group=CFG.model_name,
#                 job_type="train")

# Utils

In [ ]:
# ====================================================
# Image Utils
# ====================================================
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)


def build_masks(df_train, image_id, input_shape):
    height, width = input_shape
    labels = df_train[df_train["id"] == image_id]["annotation"].tolist()
    mask = np.zeros((height, width))
    for label in labels:
        mask += rle_decode(label, shape=(height, width))
    mask = mask.clip(0, 1)
    return mask

# Dataset Generator

In [ ]:
# ====================================================
# Dataset generator
# ====================================================
class DatasetRetriever(Dataset):
    def __init__(self, df, 
                 base_path:str, 
                 image_size:list, 
                 mean:int, std:int
                ):
        
        self.df = df
        self.base_path = base_path
        self.image_size = image_size
        self.mean = mean 
        self.std = std
        self.gb = self.df.groupby('id')
        self.image_ids = df.id.unique().tolist()
        
        # Image augmentations
        self.transforms = A.Compose([
                    A.Resize(self.image_size[0], self.image_size[1]),
                    A.Normalize(mean=self.mean, std=self.std, p=1), 
                    A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5),
                    ToTensorV2()
                ])
    
    def __len__(self):
        return len(self.image_ids)
        
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        df = self.gb.get_group(image_id)
        
        annotations = df['annotation'].tolist()
        image_path = os.path.join(self.base_path, image_id + ".png")
        
        image = cv2.imread(image_path)
        
        mask = build_masks(self.df, image_id, input_shape=(520, 704))
        mask = (mask >= 1).astype('float32')
        augmented = self.transforms(image=image, mask=mask)
        
        image = augmented['image']
        mask = augmented['mask']
        
        return image, mask.reshape((1, self.image_size[0], self.image_size[1]))


In [ ]:
# ====================================================
# Custom Loss for competition metric | from:https://www.kaggle.com/julian3833/sartorius-starter-baseline-torch-u-net
# ====================================================

def dice_loss(input, target):
    input = torch.sigmoid(input)
    smooth = 1.0
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    return ((2.0 * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))

class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))
        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()
        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        return loss.mean()

    
class MixedLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.focal = FocalLoss(gamma)

    def forward(self, input, target):
        loss = self.alpha*self.focal(input, target) - torch.log(dice_loss(input, target))
        return loss.mean()    
    

## GroupKFold

In [ ]:
df = pd.read_csv(CFG.train_df_path)
gkf  = GroupKFold(n_splits = CFG.n_fold)
df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(df, groups = df.id.tolist())):
    df.loc[val_idx, 'fold'] = fold

# Training loop

In [ ]:
def train_fn(df):
    print(f'CONFGI:\n{CFG}')
    for fold in range(CFG.n_fold):
        if fold in CFG.trn_fold:
            print(f'='*25,'Fold: ',fold,'='*25)

            valid_df = df.loc[df['fold'] == fold]
            train_df = df.loc[df['fold'] != fold]

            train_dataset = DatasetRetriever(
                                    df=train_df,
                                    base_path=CFG.train_base,
                                    image_size=CFG.image_size,
                                    mean=timm.data.IMAGENET_DEFAULT_MEAN,
                                    std=timm.data.IMAGENET_DEFAULT_STD)

            valid_dataset = DatasetRetriever(
                                    df=valid_df,
                                    base_path=CFG.train_base,
                                    image_size=CFG.image_size,
                                    mean=timm.data.IMAGENET_DEFAULT_MEAN,
                                    std=timm.data.IMAGENET_DEFAULT_STD)        

            train_loader = DataLoader(train_dataset, 
                                      batch_size=CFG.batch_size,
                                      sampler=RandomSampler(train_dataset), 
                                      num_workers=CFG.num_workers, drop_last=True)

            valid_loader = DataLoader(valid_dataset, 
                                      batch_size=CFG.batch_size, 
                                      sampler=SequentialSampler(valid_dataset), 
                                      num_workers=CFG.num_workers, drop_last=False)

            print('TRAIN: {} | VALID: {}'.format(len(train_loader.dataset), len(valid_loader.dataset)))

            # Define the model:
            model = smp.FPN(f'{CFG.encoder_name}', 
                            encoder_weights='noisy-student', 
                            activation=None)
            model.to(CFG.device)

            criterion = MixedLoss(10.0, 2.0)

            optimizer = torch.optim.Adam(model.parameters(), lr=CFG.lr)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, CFG.epochs-1)
            scaler = torch.cuda.amp.GradScaler()
            checkpoint = f'effb3_fpn-{fold}-fold_best.pth'

            count = 0
            best_epoch = 0

            for epoch in range(CFG.epochs):
                print('Epoch: {}'.format(epoch))
                model.train()

                loop = tqdm(train_loader)
                for images, masks in loop:
                    images=images.to(CFG.device)
                    masks =masks.to(CFG.device)

                    optimizer.zero_grad()

                    with torch.cuda.amp.autocast():
                        outputs = model(images)
                        loss = criterion(outputs, masks)

                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

                    loop.set_description(f'Epoch : {epoch}/{CFG.epochs} | LOSS:{loss} | LR:{optimizer.param_groups[0]["lr"]}')


                # Validation loop 
                model.eval()
                for images, masks, in tqdm(valid_loader):
                    images = images.to(CFG.device)
                    masks = masks.to(CFG.device)
                    with torch.cuda.amp.autocast(), torch.no_grad():
                        outputs = model(images)
                        val_loss = criterion(outputs, masks)
                print('End of epoch. Val loss: {}'.format(val_loss))


                if epoch == 0:
                    best_val_loss = val_loss
                    print('Saving the model...')
                    torch.save(model.state_dict(), checkpoint)
                if epoch != 0:
                    if val_loss < best_val_loss:
                        print('Saving the model!')
                        torch.save(model.state_dict(), checkpoint)
                        best_val_loss = val_loss
                    else:
                        best_val_loss = best_val_loss
                scheduler.step()


In [ ]:
if __name__ == '__main__':
    train_fn(df=df)

### Thank you!
That's pretty much it for now. I'll be working on inference notebook and Wandb will be usable soon. Pleas `upvote` if you found this notebook helpful!